In [ ]:
import sys
!{sys.executable} -m pip install pandas pybaseball

In [1]:
import pandas as pd
from pybaseball import team_pitching, pitching_stats, playerid_reverse_lookup

# Collecting baseball data

This model will be using the following statistics (for each of home and away teams) in each game:

* Team name
* Team xwOBA differential
* Lineup xwOBA (for the 9 hitters in the starting lineup)
* Team xFIP
* Starting pitcher xFIP
* Win percentage in 1-run games
* Season run differential

To predict the following outcomes:

* Runs scored
* ... and therefore, the game's winner

All numerical data will be from the past 3 seasons.

Data are provided by Retrosheet, Baseball Savant, and FanGraphs (using pybaseball).

### Team code information

Since each data source uses different team codes (for example, the Chicago White Sox are represented by "CHA" in Retrosheet, "CWS" in Baseball Savant, and "CHW" in FanGraphs), I created some helpful dictionaries to convert from one style to the other.

In [2]:
# convert team codes from retrosheet to baseball savant style

rs_to_bs = {
    'HOU': 'HOU',
    'TEX': 'TEX',
    'ANA': 'LAA',
    'OAK': 'OAK',
    'SEA': 'SEA',

    'KCA': 'KC',
    'DET': 'DET',
    'CLE': 'CLE',
    'MIN': 'MIN',
    'CHA': 'CWS',

    'NYA': 'NYY',
    'BOS': 'BOS',
    'TBA': 'TB',
    'TOR': 'TOR',
    'BAL': 'BAL',
    
    'LAN': 'LAD',
    'SDN': 'SD',
    'COL': 'COL',
    'ARI': 'AZ',
    'SFN': 'SF',
    
    'MIL': 'MIL',
    'PIT': 'PIT',
    'SLN': 'STL',
    'CIN': 'CIN',
    'CHN': 'CHC',

    'NYN': 'NYM',
    'ATL': 'ATL',
    'PHI': 'PHI',
    'WAS': 'WSH',
    'MIA': 'MIA',
}

In [3]:
# convert team codes from fangraphs to baseball savant style

fg_to_bs = {
    'HOU': 'HOU',
    'TEX': 'TEX',
    'LAA': 'LAA',
    'OAK': 'OAK',
    'SEA': 'SEA',

    'KCR': 'KC',
    'DET': 'DET',
    'CLE': 'CLE',
    'MIN': 'MIN',
    'CHW': 'CWS',

    'NYY': 'NYY',
    'BOS': 'BOS',
    'TBR': 'TB',
    'TOR': 'TOR',
    'BAL': 'BAL',
    
    'LAD': 'LAD',
    'SDP': 'SD',
    'COL': 'COL',
    'ARI': 'AZ',
    'SFG': 'SF',
    
    'MIL': 'MIL',
    'PIT': 'PIT',
    'STL': 'STL',
    'CIN': 'CIN',
    'CHC': 'CHC',

    'NYM': 'NYM',
    'ATL': 'ATL',
    'PHI': 'PHI',
    'WSN': 'WSH',
    'MIA': 'MIA',
}

In [4]:
# convert team codes from retrosheet to fangraphs style

rs_to_fg = {
    'HOU': 'HOU',
    'TEX': 'TEX',
    'ANA': 'LAA',
    'OAK': 'OAK',
    'SEA': 'SEA',

    'KCA': 'KCR',
    'DET': 'DET',
    'CLE': 'CLE',
    'MIN': 'MIN',
    'CHA': 'CHW',

    'NYA': 'NYY',
    'BOS': 'BOS',
    'TBA': 'TBR',
    'TOR': 'TOR',
    'BAL': 'BAL',
    
    'LAN': 'LAD',
    'SDN': 'SDP',
    'COL': 'COL',
    'ARI': 'ARI',
    'SFN': 'SFG',
    
    'MIL': 'MIL',
    'PIT': 'PIT',
    'SLN': 'STL',
    'CIN': 'CIN',
    'CHN': 'CHC',

    'NYN': 'NYM',
    'ATL': 'ATL',
    'PHI': 'PHI',
    'WAS': 'WSN',
    'MIA': 'MIA',
}

### Player code information

Because player names can be entered differently in different data sources (especially regarding the use of accent markings, suffixes, abbreviated names, and hyphenated names), I created a csv file containing each player's name and IDs for each data source.

In [5]:
### DON'T RUN AGAIN!!!

player_info = {
    'last_name': [],
    'first_name': [],
    'mlbam_id': [],
    'retrosheet_id': [],
    'fangraphs_id': []
}


player_xwoba_df = pd.read_csv('../data/player_xwoba.csv')
player_xfip_df = pd.read_csv('../data/player_xfip.csv')


for index, row in player_xwoba_df.iterrows():
    player_id = row['player_id']
    player = playerid_reverse_lookup([player_id], key_type='mlbam').iloc[0]
    
    player_info['last_name'].append(player['name_last'])
    player_info['first_name'].append(player['name_first'])
    player_info['mlbam_id'].append(player['key_mlbam'])
    player_info['retrosheet_id'].append(player['key_retro'])
    player_info['fangraphs_id'].append(player['key_fangraphs'])


for index, row in player_xfip_df.iterrows():
    player_id = row['IDfg']
    player = playerid_reverse_lookup([player_id], key_type='fangraphs').iloc[0]
    
    player_info['last_name'].append(player['name_last'])
    player_info['first_name'].append(player['name_first'])
    player_info['mlbam_id'].append(player['key_mlbam'])
    player_info['retrosheet_id'].append(player['key_retro'])
    player_info['fangraphs_id'].append(player['key_fangraphs'])

Gathering player lookup table. This may take a moment.


In [15]:
player_info_df = pd.DataFrame(player_info)
player_info_df.to_csv('../data/player_info.csv', index=False)

## Calculating team metrics

The following code will calculate the win percentage in 1-run games and the season run differential after traversing through game logs from Retrosheet. The compiled data is then saved to a csv file for later use.

In [31]:
### DON'T RUN AGAIN!!!

team_metrics = {
    'team': [],
    'year': [],
    'close_games': [],
    'close_win_pct': [],
    'run_diff': []
}


for year in range(2015, 2024): # collect data for each individual season from 2015-2023
    gamelog_df = pd.read_csv(f'../data/gamelogs/gl{year}.txt')

    for team in rs_to_bs:
        total_rs = 0
        total_ra = 0
        close_wins = 0
        close_losses = 0
        
        for index, row in gamelog_df.iterrows():
            if row['away_team'] != team and row['home_team'] != team:
                continue

            if row['away_team'] == team:
                rs = row['away_score']
                ra = row['home_score']
            
            if row['home_team'] == team:
                rs = row['home_score']
                ra = row['away_score']
                
            total_rs += rs
            total_ra += ra

            if abs(rs - ra) <= 1:
                if rs > ra:
                    close_wins += 1
                
                if ra > rs:
                    close_losses += 1

        team_metrics['team'].append(rs_to_bs[team])
        team_metrics['year'].append(year)
        team_metrics['close_games'].append(close_wins + close_losses)
        team_metrics['close_win_pct'].append(close_wins / (close_wins + close_losses))
        team_metrics['run_diff'].append(total_rs - total_ra)

In [33]:
team_metrics_df = pd.DataFrame(team_metrics)
team_metrics_df.to_csv('../data/team_metrics.csv', index=False)

## Calculating batting data

The following code will calculate the team xwOBA differential after traversing through the data for team xwOBA and team xwOBA against from Baseball Savant. The compiled data is then saved to a csv file for later use.

In [8]:
### DON'T RUN AGAIN!!!

team_xwoba_diff = {
    'team': [],
    'year': [],
    'xwoba_diff': [],
}


team_xwoba_df = pd.read_csv('../data/team_xwoba.csv')
team_xwoba_against_df = pd.read_csv('../data/team_xwoba_against.csv')


for index1, row1 in team_xwoba_df.iterrows():
    for index2, row2 in team_xwoba_against_df.iterrows():
        if row1['player_name'] != row2['player_name'] or row1['year'] != row2['year']:
            continue

        team_xwoba_diff['team'].append(row1['player_name'])
        team_xwoba_diff['year'].append(row1['year'])
        team_xwoba_diff['xwoba_diff'].append(row1['xwoba'] - row2['xwoba'])

In [10]:
team_xwoba_diff_df = pd.DataFrame(team_xwoba_diff)
team_xwoba_diff_df.to_csv('../data/team_xwoba_diff.csv', index=False)

## Compiling pitching data

The following code will retrieve the data for team and player xFIP from FanGraphs, using pybaseball. The compiled data is then saved to a csv file for later use.

https://github.com/jldbc/pybaseball

In [9]:
### DON'T RUN AGAIN!!!

team_xfip_df = team_pitching(2015, 2023)
team_xfip_df.to_csv('../data/team_xfip.csv', index=False)

player_xfip_df = pitching_stats(2015, 2023, qual=20)
player_xfip_df.to_csv('../data/player_xfip.csv', index=False)

## Compiling all of the data

The following code will compile all of the data into one big csv file containing information about each game from 2017 to 2023.

In [6]:
team_xwoba_diff_df = pd.read_csv('../data/team_xwoba_diff.csv')
player_xwoba_df = pd.read_csv('../data/player_xwoba.csv')
team_xfip_df = pd.read_csv('../data/team_xfip.csv')
player_xfip_df = pd.read_csv('../data/player_xfip.csv')
team_metrics_df = pd.read_csv('../data/team_metrics.csv')
player_info_df = pd.read_csv('../data/player_info.csv')

In [7]:
# NO WEIGHTS ADDED YET!!!

def get_team_xwoba_diff(year, team):
    data_year1 = team_xwoba_diff_df['xwoba_diff'].loc[team_xwoba_diff_df['team'] == team].loc[team_xwoba_diff_df['year'] == year - 2].iloc[0]
    data_year2 = team_xwoba_diff_df['xwoba_diff'].loc[team_xwoba_diff_df['team'] == team].loc[team_xwoba_diff_df['year'] == year - 1].iloc[0]
    data_year3 = team_xwoba_diff_df['xwoba_diff'].loc[team_xwoba_diff_df['team'] == team].loc[team_xwoba_diff_df['year'] == year].iloc[0]
    
    return (data_year1 + data_year2 + data_year3) / 3

In [8]:
# NO WEIGHTS ADDED YET!!!

def get_lineup_xwoba(year, lineup):
    total_xwoba = 0
    
    for hitter in lineup:
        mlbam_id = player_info_df['mlbam_id'].loc[player_info_df['retrosheet_id'] == hitter]

        if len(mlbam_id) == 0:
            total_xwoba += 0.317 # league average in 2024; change later
            continue
        else:
            mlbam_id = mlbam_id.iloc[0]
        
        data_year1 = player_xwoba_df['xwoba'].loc[player_xwoba_df['player_id'] == mlbam_id].loc[player_xwoba_df['year'] == year - 2]

        if len(data_year1) == 0:
            data_year1 = 0.317
        else:
            data_year1 = data_year1.iloc[0]

        data_year2 = player_xwoba_df['xwoba'].loc[player_xwoba_df['player_id'] == mlbam_id].loc[player_xwoba_df['year'] == year - 1]

        if len(data_year2) == 0:
            data_year2 = 0.317
        else:
            data_year2 = data_year2.iloc[0]

        data_year3 = player_xwoba_df['xwoba'].loc[player_xwoba_df['player_id'] == mlbam_id].loc[player_xwoba_df['year'] == year]

        if len(data_year3) == 0:
            data_year3 = 0.317
        else:
            data_year3 = data_year3.iloc[0]
            
        total_xwoba += (data_year1 + data_year2 + data_year3) / 3

    return total_xwoba / 9

In [9]:
# NO WEIGHTS ADDED YET!!!

def get_team_xfip(year, team):
    data_year1 = team_xfip_df['xFIP'].loc[team_xfip_df['Team'] == team].loc[team_xfip_df['Season'] == year - 2].iloc[0]
    data_year2 = team_xfip_df['xFIP'].loc[team_xfip_df['Team'] == team].loc[team_xfip_df['Season'] == year - 1].iloc[0]
    data_year3 = team_xfip_df['xFIP'].loc[team_xfip_df['Team'] == team].loc[team_xfip_df['Season'] == year].iloc[0]

    return (data_year1 + data_year2 + data_year3) / 3

In [10]:
# NO WEIGHTS ADDED YET!!!

def get_starter_xfip(year, pitcher):
    fangraphs_id = player_info_df['fangraphs_id'].loc[player_info_df['retrosheet_id'] == pitcher]

    if len(fangraphs_id) == 0:
        return 4.26 # league average in 2023; change later
    else:
        fangraphs_id = fangraphs_id.iloc[0]
    
    data_year1 = player_xfip_df['xFIP'].loc[player_xfip_df['IDfg'] == fangraphs_id].loc[player_xfip_df['Season'] == year - 2]

    if len(data_year1) == 0:
        data_year1 = 4.26
    else:
        data_year1 = data_year1.iloc[0]
        
    data_year2 = player_xfip_df['xFIP'].loc[player_xfip_df['IDfg'] == fangraphs_id].loc[player_xfip_df['Season'] == year - 1]

    if len(data_year2) == 0:
        data_year2 = 4.26 # league average in 2023; change later
    else:
        data_year2 = data_year2.iloc[0]
    
    data_year3 = player_xfip_df['xFIP'].loc[player_xfip_df['IDfg'] == fangraphs_id].loc[player_xfip_df['Season'] == year]

    if len(data_year3) == 0:
        data_year3 = 4.26 # league average in 2023; change later
    else:
        data_year3 = data_year3.iloc[0]
    
    return (data_year1 + data_year2 + data_year3) / 3

In [11]:
# NO WEIGHTS ADDED YET!!!

def get_close_win_pct(year, team):
    data_year1 = team_metrics_df['close_win_pct'].loc[team_metrics_df['team'] == team].loc[team_metrics_df['year'] == year - 2].iloc[0]
    data_year2 = team_metrics_df['close_win_pct'].loc[team_metrics_df['team'] == team].loc[team_metrics_df['year'] == year - 1].iloc[0]
    data_year3 = team_metrics_df['close_win_pct'].loc[team_metrics_df['team'] == team].loc[team_metrics_df['year'] == year].iloc[0]
    
    return (data_year1 + data_year2 + data_year3) / 3

In [12]:
# NO WEIGHTS ADDED YET!!!

def get_run_diff(year, team):
    data_year1 = team_metrics_df['run_diff'].loc[team_metrics_df['team'] == team].loc[team_metrics_df['year'] == year - 2].iloc[0]
    data_year2 = team_metrics_df['run_diff'].loc[team_metrics_df['team'] == team].loc[team_metrics_df['year'] == year - 1].iloc[0]
    data_year3 = team_metrics_df['run_diff'].loc[team_metrics_df['team'] == team].loc[team_metrics_df['year'] == year].iloc[0]
    
    return (data_year1 + data_year2 + data_year3) / 3

In [13]:
# enter individual game data into the dictionary

def write_game_data(data, row, year):
    data['away_team'].append(rs_to_bs[row['away_team']])
    data['away_score'].append(row['away_score'])
    data['away_team_xwoba_diff'].append(get_team_xwoba_diff(year, rs_to_bs[row['away_team']]))
    data['away_lineup_xwoba'].append(get_lineup_xwoba(year, [row['away_1_id'], row['away_2_id'], row['away_3_id'],
                                                        row['away_4_id'], row['away_5_id'], row['away_6_id'],
                                                        row['away_7_id'], row['away_8_id'], row['away_9_id']]))
    data['away_team_xfip'].append(get_team_xfip(year, rs_to_fg[row['away_team']]))
    data['away_starter_xfip'].append(get_starter_xfip(year, row['away_starter_id']))
    data['away_close_win_pct'].append(get_close_win_pct(year, rs_to_bs[row['away_team']]))
    data['away_run_diff'].append(get_run_diff(year, rs_to_bs[row['away_team']]))
    data['home_team'].append(rs_to_bs[row['home_team']])
    data['home_score'].append(row['home_score'])
    data['home_team_xwoba_diff'].append(get_team_xwoba_diff(year, rs_to_bs[row['home_team']]))
    data['home_lineup_xwoba'].append(get_lineup_xwoba(year, [row['home_1_id'], row['home_2_id'], row['home_3_id'],
                                                    row['home_4_id'], row['home_5_id'], row['home_6_id'],
                                                    row['home_7_id'], row['home_8_id'], row['home_9_id']]))
    data['home_team_xfip'].append(get_team_xfip(year, rs_to_fg[row['home_team']]))
    data['home_starter_xfip'].append(get_starter_xfip(year, row['home_starter_id']))
    data['home_close_win_pct'].append(get_close_win_pct(year, rs_to_bs[row['home_team']]))
    data['home_run_diff'].append(get_run_diff(year, rs_to_bs[row['home_team']]))

In [14]:
# training data - game results from 2017-2022

train_data = {
    'away_team': [],
    'away_score': [],
    'away_team_xwoba_diff': [],
    'away_lineup_xwoba': [],
    'away_team_xfip': [],
    'away_starter_xfip': [],
    'away_close_win_pct': [],
    'away_run_diff': [],
    'home_team': [],
    'home_score': [],
    'home_team_xwoba_diff': [],
    'home_lineup_xwoba': [],
    'home_team_xfip': [],
    'home_starter_xfip': [],
    'home_close_win_pct': [],
    'home_run_diff': []
}

In [16]:
for year in range(2017, 2024):
    gamelog_df = pd.read_csv(f'../data/gamelogs/gl{year}.txt')
    
    for index, row in gamelog_df.iterrows():
        write_game_data(train_data, row, year)
        

game_data_df = pd.DataFrame(train_data)
game_data_df.to_csv('../data/game_data_v1.csv', index=False)